<a href="https://colab.research.google.com/github/SpirinEgor/topic_modeling_story/blob/master/train_ctm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Tue Nov 30 17:10:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    77W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! git clone https://github.com/SpirinEgor/topic_modeling_story
%cd topic_modeling_story

fatal: destination path 'topic_modeling_story' already exists and is not an empty directory.
/content/topic_modeling_story


In [3]:
! git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/SpirinEgor/topic_modeling_story
   1d5541a..5d4bfab  master     -> origin/master
Updating 1d5541a..5d4bfab
Fast-forward
 src/dataset.py | 4 +---
 1 file changed, 1 insertion(+), 3 deletions(-)


In [9]:
! wget https://voudy-data.s3.eu-north-1.amazonaws.com/pycharm_issues.json

--2021-11-30 15:52:42--  https://voudy-data.s3.eu-north-1.amazonaws.com/pycharm_issues.json
Resolving voudy-data.s3.eu-north-1.amazonaws.com (voudy-data.s3.eu-north-1.amazonaws.com)... 52.95.169.44
Connecting to voudy-data.s3.eu-north-1.amazonaws.com (voudy-data.s3.eu-north-1.amazonaws.com)|52.95.169.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27987977 (27M) [application/json]
Saving to: ‘pycharm_issues.json’

pycharm_issues.json 100%[===================>]  26.69M  54.0MB/s    in 0.5s    

2021-11-30 15:52:43 (54.0 MB/s) - ‘pycharm_issues.json’ saved [27987977/27987977]



In [11]:
%%capture
! pip install contextualized-topic-models pyldavis

In [16]:
import pickle

import nltk
import pyLDAvis as vis

from src.dataset import YouTrackIssueDataset
from src.ctm import ContextualizedTopicModeling

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
dataset = YouTrackIssueDataset("pycharm_issues.json")
len(dataset)

22014

In [7]:
%%time
ctm = ContextualizedTopicModeling(dataset, "paraphrase-distilroberta-base-v2")

Found 22014 documents
Preprocessing documents...


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Done in 0.959435224533081 s


Batches:   0%|          | 0/110 [00:00<?, ?it/s]

CPU times: user 47 s, sys: 1.73 s, total: 48.7 s
Wall time: 54.1 s


In [8]:
ctm.fit(contextual_size=768, n_topics=50, num_epochs=50)

Epoch: [50/50]	 Seen Samples: [1098400/1098400]	Train Loss: 92.40386014356287	Time: 0:00:09.474048: : 50it [07:57,  9.56s/it]
Sampling: [20/20]: : 20it [02:44,  8.21s/it]


In [13]:
lda_vis_data = ctm.model.get_ldavis_data_format(ctm._tp.vocab, ctm._train_dataset, 30)

Sampling: [30/30]: : 30it [04:08,  8.29s/it]


In [14]:
vis_pd = vis.prepare(**lda_vis_data)
vis.display(vis_pd)

In [15]:
ctm.save("cmt.pkl")

In [18]:
with open("vis_data.pkl", "wb") as f:
    pickle.dump(lda_vis_data, f)